In [ ]:
!pip install datasets transformers optuna safetensors

import torch
import torch.nn as nn
import torch.nn.functional as F
from concurrent.futures import ThreadPoolExecutor
from transformers import AutoTokenizer, Trainer, TrainingArguments, DebertaV2ForSequenceClassification
from datasets import load_dataset, load_metric
import time

class AdvancedQuantumInspiredModel(nn.Module):
    def __init__(self, deberta_model):
        super(AdvancedQuantumInspiredModel, self).__init__()
        self.deberta_model = deberta_model
        self.conv1 = nn.Conv1d(768, 768, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(768, 768, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(768 * 128, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.deberta_model(input_ids=input_ids, attention_mask=attention_mask)
        x = outputs.hidden_states[-1].transpose(1, 2)

        def process_path1(x):
            return F.relu(self.conv1(x))

        def process_path2(x):
            return F.relu(self.conv2(x))

        with ThreadPoolExecutor(max_workers=2) as executor:
            futures = [executor.submit(process_path1, x), executor.submit(process_path2, x)]
            results = [f.result() for f in futures]

        x = (results[0] + results[1]) / 2

        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))

        if torch.rand(1).item() > 0.5:
            x = x * torch.rand_like(x)

        logits = self.fc2(x)

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.fc2.out_features), labels.view(-1))
            return loss, logits
        else:
            return logits

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
deberta_model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-small', output_hidden_states=True)
quantum_model = AdvancedQuantumInspiredModel(deberta_model)

dataset = load_dataset('super_glue', 'boolq')
metric = load_metric('super_glue', 'boolq')

def preprocess_function(examples):
    return tokenizer(examples['question'], examples['passage'], truncation=True, padding='max_length', max_length=128)

encoded_dataset = dataset.map(preprocess_function, batched=True)

train_dataset = encoded_dataset['train'].select(range(50))
eval_dataset = encoded_dataset['validation'].select(range(50))

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy='epoch'
)

class AdvancedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get('labels')
        outputs = model(inputs['input_ids'], inputs['attention_mask'], labels=labels)
        loss = outputs[0]
        return (loss, outputs) if return_outputs else loss

trainer = AdvancedTrainer(
    model=quantum_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=lambda p: metric.compute(predictions=torch.argmax(torch.tensor(p.predictions), axis=1), references=p.label_ids),
)

print("Training and evaluating Quantum-Inspired model...")
trainer.model = quantum_model
trainer.train()

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=128)

quantum_model.eval()
start_time = time.time()
with torch.no_grad():
    outputs = quantum_model(inputs['input_ids'], inputs['attention_mask'])
end_time = time.time()
quantum_inference_time = end_time - start_time
print(f"Quantum-Inspired Model Inference Time: {quantum_inference_time} seconds")

print("Evaluating the quantum-inspired model...")
trainer.model = quantum_model
results = trainer.evaluate()

print("Model results:", results)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Training and evaluating Quantum-Inspired model...


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.831349,0.200000


Quantum-Inspired Model Inference Time: 0.43195676803588867 seconds
Evaluating the quantum-inspired model...


Model results: {'eval_loss': 0.8059699535369873, 'eval_accuracy': 0.24, 'eval_runtime': 15.6792, 'eval_samples_per_second': 3.189, 'eval_steps_per_second': 0.446, 'epoch': 1.0}


In [ ]:
!pip install transformers

import torch
from transformers import AutoTokenizer, DebertaV2ForSequenceClassification
import time

tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v3-small')
model = DebertaV2ForSequenceClassification.from_pretrained('microsoft/deberta-v3-small')

input_text = "Hello, how are you?"
inputs = tokenizer(input_text, return_tensors='pt', padding='max_length', max_length=128)

model.eval()
start_time = time.time()
with torch.no_grad():
    outputs = model(**inputs)
end_time = time.time()

inference_time = end_time - start_time
print(f"Baseline DeBERTa-v3 Model Inference Time: {inference_time} seconds")

# Print model outputs (logits)
print(f"Model outputs: {outputs.logits}")


/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Baseline DeBERTa-v3 Model Inference Time: 0.6665582656860352 seconds
Model outputs: tensor([[-0.1372,  0.1318]])
